<a href="https://colab.research.google.com/github/mineptune/Text_Toxicity_Moderation/blob/main/Text_Toxicity_Moderation_Model_With_Gradio_FIXED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Text Toxicity Moderation Model
**Author:** [Your Name]  
**Environment:** Jupyter Notebook  
**Objective:** Detect whether a piece of text is toxic or non-toxic using NLP techniques.


## 🧭 1. Introduction
### Objective
The goal of this notebook is to develop a model that can automatically identify toxic comments — such as hate speech, insults, or harassment — using natural language processing (NLP).

### Why It Matters
Text toxicity detection is critical for moderating online platforms, maintaining community standards, and reducing online abuse.

### Approach
We'll explore two approaches:
1. **Baseline Model:** TF-IDF + Logistic Regression  
2. **Advanced Model:** Pre-trained BERT (Hugging Face)


In [ ]:
# ⚙️ Setup and Imports
!pip install pandas numpy scikit-learn matplotlib seaborn transformers torch gradio SpeechRecognition --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# 📥 Load the Dataset
data = {
    'text': [
        "I love this product!",
        "You are such an idiot.",
        "What a wonderful day.",
        "This is the worst thing ever.",
        "I hate you so much.",
        "Great job on the project!"
    ],
    'label': [0, 1, 0, 1, 1, 0]
}
df = pd.DataFrame(data)
df.head()


In [ ]:
# 🔍 Exploratory Data Analysis (EDA)
print(df['label'].value_counts())
sns.countplot(x='label', data=df)
plt.title("Distribution of Toxic vs Non-Toxic Comments")
plt.show()


In [ ]:
# 🧹 Data Preprocessing
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    return text.strip()

df['clean_text'] = df['text'].apply(clean_text)
df.head()


In [ ]:
# 🧩 Baseline Model – TF-IDF + Logistic Regression
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['label'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = LogisticRegression()
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)

print(classification_report(y_test, y_pred))


In [ ]:
# 🤖 Advanced Model – Pre-trained BERT (Hugging Face)
from transformers import pipeline

toxic_model = pipeline("text-classification", model="unitary/toxic-bert")

examples = [
    "I hate this product so much!",
    "You're amazing and kind.",
    "Go die in a hole."
]
for e in examples:
    print(e, "→", toxic_model(e))


In [ ]:
# 📊 Model Evaluation and Examples
def predict_toxicity(text):
    cleaned = clean_text(text)
    vec = vectorizer.transform([cleaned])
    pred = model.predict(vec)[0]
    return "Toxic" if pred == 1 else "Non-Toxic"

sample_texts = [
    "I can't stand you.",
    "You're doing great work.",
    "This app sucks."
]

for t in sample_texts:
    print(f"{t} → {predict_toxicity(t)}")


## 💡 Conclusion & Future Work
### Results Summary
- Logistic Regression (TF-IDF) achieved decent baseline performance.
- BERT-based model (pretrained) delivers higher contextual accuracy.

### Future Improvements
- Fine-tune BERT on a custom dataset.
- Add explainability (e.g., SHAP values).
- Integrate into an API for real-time moderation.

### Ethical Considerations
- Ensure fairness: avoid bias against dialects, slang, or specific groups.
- Consider privacy, transparency, and accountability.


In [ ]:
# 🎙️ Interactive Gradio Interface (Text + Voice)
import gradio as gr
from transformers import pipeline
import speech_recognition as sr

# Load pre-trained model
toxic_model = pipeline("text-classification", model="unitary/toxic-bert")

def classify_toxicity(text, audio=None):
    # If audio provided, use speech recognition to extract text
    if audio is not None:
        try:
            recognizer = sr.Recognizer()
            with sr.AudioFile(audio) as source:
                audio_data = recognizer.record(source)
                text_from_audio = recognizer.recognize_google(audio_data)
                text = text_from_audio
        except Exception as e:
            return f"⚠️ Error recognizing speech: {e}"
    if not text or text.strip() == "":
        return "⚠️ Please enter or speak a sentence."
    result = toxic_model(text)[0]
    label = result['label']
    score = round(result['score'], 3)
    return f"🧾 Input: {text}\n\nPrediction: {label}\nConfidence: {score}"

# Launch Gradio Interface
interface = gr.Interface(
    fn=classify_toxicity,
    inputs=[
        gr.Textbox(label="Type a sentence here"),
        gr.Audio(source="microphone", type="filepath", label="🎤 Or speak your sentence (speech-to-text)")
    ],
    outputs="text",
    title="🧠 Real-Time Text Toxicity Detector",
    description="Type or speak a sentence and see if it's toxic using a fine-tuned NLP model.",
    allow_flagging="never"
)

interface.launch()
